In [1]:
import math
import numpy as np
from astropy.io.votable import parse_single_table
from astropy.coordinates import SkyCoord
from astropy.table import Column
import astropy.units as u
from astropy.io import ascii
import matplotlib.pyplot as plt

## Produce a latex table extract

### Source list

In [2]:
# Read the sources
parent_folder = ''
spec_table = parse_single_table(parent_folder+'gaskap_smc_spectra.vot').to_table()
spec_table

id,comp_name,ra,dec,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,has_mw_abs,has_other_abs,semi_maj_axis,semi_min_axis,pa,nh_gass,noise_flag,beam_power,nh_uncorr,e_nh_uncorr,nh_corr,r_hi,f_int,mean_ts,ew,num_chan
,,deg,deg,,mJy/beam,mJy/beam,,,,,,,km / s,,,,,arcsec,arcsec,deg,1 / cm2,,,1 / cm2,1 / cm2,1 / cm2,,,,km / s,
int64,str32,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,float64,float64,float64,float64,bool,float32,float64,float64,float64,float64,float64,float64,float32,int64
1,J004725-712727,11.85596,-71.457707,B,15.015000343322754,0.0001577413873746991,0.20037295013116518,1.1125783920288086,1.0923846748301125,1.5317274332046509,43.0,0.41914910078048706,96.17224896304,0.869528591632843,0.3925101382105877,False,False,3.82400016784668,3.2720001220703128,133.6300048828125,6.174081421437779e+20,False,1.0359205,5.365187210776425e+20,1.986559023022372e+19,5.917234854795377e+20,1.1028943860356109,1.1028943860356109,49.397795872342485,7.715428,40
5,J002620-743741,6.586833,-74.628167,C,15.201000213623047,0.00014202001038938762,0.26400873561229615,1.9746829271316528,1.4209718128364035,1.815657138824463,67.0,-0.15902580320835114,132.02632915333402,5.0,3.8316940181297285,False,False,4.056000137329102,3.3919998168945313,135.64999389648438,1.3315663738275294e+21,False,0.88999236,1.662447479255305e+21,4.117855118886791e+19,1.8430605679293883e+21,1.1086428840175986,1.1086428840175986,125.01606621274738,13.0891905,75
9,J011635-712604,19.148794,-71.434582,C,15.333000183105469,0.0001255630166269839,0.21174872823938362,1.6042497158050537,0.7604905305141251,1.9112515449523926,74.0,0.30700182914733887,185.81443163211898,1.1809015274047852,0.6544683758159434,False,False,3.8040000915527346,3.2080001831054688,142.02000427246094,1.227406986785527e+21,False,1.0161469,9.243698933001397e+20,2.9086163229354488e+19,9.876735081156763e+20,1.0684829907100641,1.0684829907100641,169.7816602456293,6.9796023,85
10,J010425-742733,16.10635,-74.459193,C,15.435999870300293,0.00011435524793341756,0.2508074329566693,1.681606411933899,0.8255535251261913,1.9211487770080566,78.0,0.2395423799753189,75.45870833663803,1.4290249347686768,0.7164151873632499,False,False,3.7599998474121095,3.2759998321533206,139.1699981689453,3.131320668910356e+20,False,0.95265865,--,0.0,--,--,--,0.0,0.0,0
13,J005535-723533,13.899411,-72.592574,C,15.053000450134277,8.522021817043423e-05,0.16227310227980046,1.856298565864563,2.5452221979121297,1.5236057043075562,93.0,-0.33269286155700684,113.61597766529509,5.0,3.6192534660730047,False,True,5.552000045776367,4.431999969482422,140.7100067138672,7.446596934360939e+21,False,0.9928809,7.168075769842141e+21,6.882111346062339e+19,8.849933566255248e+21,1.2346316989964163,1.2346316989964163,479.813134319267,19.196941,94
14,J012336-703232,20.902153,-70.542329,C,15.822999954223633,0.00015618808101862668,0.24878249600046518,1.6433773040771484,0.9665233841956862,1.8356764316558838,55.0,0.19229918718338013,225.31028406877002,1.64870285987854,0.8308259887378207,False,False,3.9840000152587893,3.2560001373291017,123.41999816894531,4.156697450338526e+20,False,1.0374693,2.8895810711429556e+20,1.4532738940086557e+19,2.88870179026072e+20,0.9996957064499707,0.9996957064499707,-110.38500318620588,-0.62251383,28
15,J003530-723655,8.877739,-72.615448,C,16.052000045776367,0.0001357692526653409,0.26143587233583027,1.6275757551193237,0.8721049797536266,1.8693827390670776,72.0,0.2418069839477539,235.912981704439,1.4196155071258545,0.7330660214610306,False,False,3.808000183105469,3.2680000305175785,134.50999450683594,9.376269000777239e+20,False,0.9938403,8.317796077639564e+20,2.5192293467713835e+19,8.449818303161053e+20,1.0158722604268213,1.0158722604268213,-388.8760488258296,1.6447271,54
17,J010524-731518,16.350785,-73.255016,B,15.25100040435791,0.00016707086469978095,0.18895516231333526,1.185164451599121,0.945

In [3]:
tau = []
for row in spec_table:
    if row['min_opacity'] < 0:
        min_tau = row['peak_tau'] - row['e_peak_tau']
        if min_tau < 0:
            min_tau = 0
        tau_val = '$>{:.2f}$'.format(min_tau)
    else:
        tau_val = '${:.2f}\pm{:.2f}$'.format(row['peak_tau'], row['e_peak_tau'])
    tau.append(tau_val)
spec_table.add_column(Column(data=tau, name='tau'))

In [4]:
# Select a sample of sources surrounding the sample source
central_comp_name = 'J005556-722605'
central_entry = spec_table[spec_table['comp_name']==central_comp_name]
central_entry
central_coord = SkyCoord(central_entry['ra'], central_entry['dec'])

all_coords = SkyCoord(spec_table['ra'], spec_table['dec'])
sample = all_coords.separation(central_coord) < 0.5*u.deg

In [5]:
sample_table = spec_table[sample]
sample_table.sort(['comp_name', 'ra', 'rating', 'id'])
sample_table

id,comp_name,ra,dec,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,has_mw_abs,has_other_abs,semi_maj_axis,semi_min_axis,pa,nh_gass,noise_flag,beam_power,nh_uncorr,e_nh_uncorr,nh_corr,r_hi,f_int,mean_ts,ew,num_chan,tau
,,deg,deg,,mJy/beam,mJy/beam,,,,,,,km / s,,,,,arcsec,arcsec,deg,1 / cm2,,,1 / cm2,1 / cm2,1 / cm2,,,,km / s,,
int64,str32,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,float64,float64,float64,float64,bool,float32,float64,float64,float64,float64,float64,float64,float32,int64,str13
304,J004956-723554,12.486259,-72.598535,A,81.70099639892578,0.0007445093709975481,0.033805358987482284,0.6342768669128418,4.180762669533237,1.1224292516708374,110.0,0.4881523549556732,141.6918558646755,0.7171277403831482,0.09901098194686853,True,True,4.6959999084472654,3.3520000457763675,129.13999938964844,4.815617270551048e+21,False,1.0075643,4.188194586209647e+21,7.772513771344606e+19,4.5182865736628715e+21,1.0788148641756306,1.0788148641756306,321.28181550605234,8.760021,102,$0.72\pm0.10$
357,J005218-722708,13.077894,-72.4523,B,153.1790008544922,0.0013815705664455892,0.018857461197800653,0.2662339210510254,2.8796295009519572,1.0686235427856445,89.0,0.8023896217346191,156.5886135815,0.22016097605228424,0.037043324380995585,True,True,3.9000000000000004,3.779999923706055,78.58999633789062,4.655635125623896e+21,False,0.991461,3.89142471806489e+21,6.73755691151531e+19,3.974679961063603e+21,1.021394540311219,1.021394540311219,940.156196574309,2.4065187,101,$0.22\pm0.04$
287,J005337-723144,13.406718,-72.528982,A,68.66000366210938,0.0006143004633486271,0.0469270140600002,1.1960872411727905,7.319547066539747,1.143768310546875,91.0,-0.052318915724754333,129.60935854448388,5.0,2.38775086610857,False,True,3.8200000762939457,3.284000015258789,136.85000610351562,6.105707044030256e+21,False,0.9926791,5.000882404398724e+21,9.212852963941314e+19,5.59860479200949e+21,1.1195233839302072,1.1195233839302072,232.244290358606,17.464056,99,$>2.61$
200,J005423-725257,13.597904,-72.882719,B,34.69900131225586,0.0003487507114186883,0.10469237691287128,1.5519860982894897,3.2235181842542193,1.3674628734588623,102.0,-0.18452325463294983,122.774661251606,5.0,3.322804786935989,False,True,3.9439998626708985,3.315999984741211,133.44000244140625,7.398955360997733e+21,False,0.97563636,7.549550284759109e+21,8.311898321471044e+19,8.544932457635379e+21,1.1318465518251768,1.1318465518251768,401.76954802772633,15.61166,105,$>1.68$
19,J005448-725353,13.702572,-72.898137,C,15.98799991607666,0.00015485620824620127,0.18070838215007914,2.211228370666504,2.0076773559084584,1.7351946830749512,52.0,-0.4760337173938751,165.777135904169,5.0,3.821651016570783,False,True,3.8840000152587892,3.2680000305175785,137.1300048828125,6.945996611979021e+21,False,0.9740377,7.434644351591776e+21,7.2852277593603826e+19,1.0239162822324946e+22,1.3772229494922261,1.3772229494922261,205.99460310023636,39.448772,106,$>1.18$
206,J005522-721055,13.845244,-72.182103,B,30.475000381469727,0.00017335115699097515,0.07322403302617124,0.5333656668663025,1.4700425199736775,1.2159337997436523,57.0,0.6825681328773499,109.4160502677979,0.3818929195404053,0.12241143424509149,False,False,6.783999633789063,3.7759998321533206,121.0999984741211,4.0160165519979116e+21,False,0.98892707,3.178870207305299e+21,5.345805026738624e+19,3.2346880051627415e+21,1.0175590049978036,1.0175590049978036,729.8018850061977,2.7725787,85,$0.38\pm0.12$
37,J005534-721056,13.892367,-72.182354,C,16.389999389648438,9.042841265909374e-05,0.22839327817644248,1.503857970237732,1.1814257844618428,1.6893922090530396,45.0,0.18553426861763,153.41378300906712,1.6845157146453857,1.123121176919276,False,False,4.511999893188476,3.9360000610351564,72.06999969482422,4.0160165519979116e+21,False,0.98933506,3.32535982601197e+21,5.443469804548521e+19,3.733034259904671e+21,1.122595585206674,1.12259558520667

In [6]:
# Select the columns to be output
sample_table.keep_columns(['id', 'comp_name', 'ra', 'dec', 'flux_peak', 'rating', 'mean_cont', 'sd_cont', 'tau', 'nh_uncorr'])
sample_table = sample_table['id', 'comp_name', 'ra', 'dec', 'flux_peak', 'rating', 'mean_cont', 'sd_cont', 'tau', 'nh_uncorr']

In [7]:
# Set up formatting of the output table

# Column titles
names = ['ID', 'Source', 'RA', 'Dec', 'Peak Flux', 'Rating', r'S$_{\rm cont}$', r'$\sigma_{\rm cont}$', r'Peak $\tau$', 'N$_{\rm HI,uncorr}$']

# Note the names need to be the ones in the table output, as defined above
formats={'Source': lambda x: x.replace('-', '$-$'), 'RA': '%.4f', 'Dec': '$%.3f$', 'Peak Flux' : '%.0f', names[6] : '%.3f', names[7] : '%.3f', 
         names[-1] : lambda x: str(round(x/1e21, 1))}

# We have to set the units manually to get the brackets around the units
units = {'RA':'(deg)', 'Dec':'(deg)', 'Peak Flux': '(mJy)', names[-1] : r'($10^{21}$ cm$^{-2}$)'}

col_align=''.join(['c' if name in ('Source', 'Rating') else 'r'  for name in names])
latextdict = {'units': units, 'tabletype': None, 'header_start': '\\hline \\hline', 
              'header_end': '\\hline', 'data_end': '\\hline'}

In [8]:
sample_table

id,comp_name,ra,dec,flux_peak,rating,mean_cont,sd_cont,tau,nh_uncorr
,,deg,deg,mJy/beam,,mJy/beam,,,1 / cm2
int64,str32,float64,float64,float64,str1,float64,float64,str13,float64
304,J004956-723554,12.486259,-72.598535,81.70099639892578,A,0.0007445093709975481,0.033805358987482284,$0.72\pm0.10$,4.188194586209647e+21
357,J005218-722708,13.077894,-72.4523,153.1790008544922,B,0.0013815705664455892,0.018857461197800653,$0.22\pm0.04$,3.89142471806489e+21
287,J005337-723144,13.406718,-72.528982,68.66000366210938,A,0.0006143004633486271,0.0469270140600002,$>2.61$,5.000882404398724e+21
200,J005423-725257,13.597904,-72.882719,34.69900131225586,B,0.0003487507114186883,0.10469237691287128,$>1.68$,7.549550284759109e+21
19,J005448-725353,13.702572,-72.898137,15.98799991607666,C,0.00015485620824620127,0.18070838215007914,$>1.18$,7.434644351591776e+21
206,J005522-721055,13.845244,-72.182103,30.475000381469727,B,0.00017335115699097515,0.07322403302617124,$0.38\pm0.12$,3.178870207305299e+21
37,J005534-721056,13.892367,-72.182354,16.389999389648438,C,9.042841265909374e-05,0.22839327817644248,$1.68\pm1.12$,3.32535982601197e+21
13,J005535-723533,13.899411,-72.592574,15.053000450134277,C,8.522021817043423e-05,0.16227310227980046,$>1.38$,7.168075769842141e+21


In [9]:

ascii.write(sample_table, 'spectra_table.tex', Writer=ascii.Latex, latexdict=latextdict, 
            formats=formats, names=names, col_align=col_align, overwrite=True)

## Absorption List

In [10]:
parent_folder = 'sb10944/v3a/'
abs_table = parse_single_table(parent_folder+'askap_absorption.vot').to_table()
abs_table



id,comp_name,abs_name,ra,dec,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,semi_maj_axis,semi_min_axis,pa,start_vel,end_vel,length,min_optical_depth,e_min_optical_depth,peak_tau,e_peak_tau,max_sigma
,,,deg,deg,,mJy/beam,mJy/beam,,,,,,arcsec,arcsec,deg,km / s,km / s,chan,,,,,
int64,str32,str32,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64
13,J005535-723533,J005535-723533_113,13.899411,-72.592574,C,15.053000450134277,8.522021817043423e-05,0.16227310227980046,1.856298565864563,2.5452221979121297,1.5236057043075562,93.0,5.552000045776367,4.431999969482422,140.7100067138672,113.61597766529509,114.6159776652694,2,-0.33269286155700684,0.2513908112211382,5.0,3.6192534660730047,5.30127913221415
13,J005535-723533,J005535-723533_116,13.899411,-72.592574,C,15.053000450134277,8.522021817043423e-05,0.16227310227980046,1.856298565864563,2.5452221979121297,1.5236057043075562,93.0,5.552000045776367,4.431999969482422,140.7100067138672,116.615977665218,117.6159776651923,2,-0.06738349795341492,0.27018215923557837,5.0,3.6913411163683785,3.9506069933546564
19,J005448-725353,J005448-725353_116,13.702572,-72.898137,C,15.98799991607666,0.00015485620824620127,0.18070838215007914,2.211228370666504,2.0076773559084584,1.7351946830749512,52.0,3.8840000152587892,3.2680000305175785,137.1300048828125,116.77713590216,118.77713590224198,3,-0.12201661616563797,0.310779098902246,5.0,3.8313270879007435,3.6103350330925146
19,J005448-725353,J005448-725353_163,13.702572,-72.898137,C,15.98799991607666,0.00015485620824620127,0.18070838215007914,2.211228370666504,2.0076773559084584,1.7351946830749512,52.0,3.8840000152587892,3.2680000305175785,137.1300048828125,163.77713590408698,167.77713590425103,5,-0.4760337173938751,0.3077864799197406,5.0,3.821651016570783,4.795641731815016
26,J010532-721331,J010532-721331_131,16.386939,-72.225495,C,16.305999755859375,0.00013356159906834363,0.20625884450647666,1.7323534488677979,1.4747387200154491,1.7000147104263306,105.0,3.7759998321533206,3.2279998779296877,138.6999969482422,131.318490585975,133.318490586057,3,-0.03233879804611206,0.24499679862438659,5.0,3.59348986463798,4.2136830499302675
33,J005014-730326,J005014-730326_152,12.561714,-73.057288,C,16.516000747680664,0.00013204875867813825,0.19689896771242069,1.833931803703308,1.7889600056779422,1.657568335533142,83.0,4.231999969482422,3.7040000915527345,124.37999725341797,152.9137555152928,153.9137555152671,2,-0.06129693612456322,0.32034791657862954,5.0,3.8616523655032515,3.3129509665138017
42,J013134-700042,J013134-700042_240,22.893692,-70.011737,B,15.736000061035156,9.92985675111413e-05,0.10793941277669811,1.12614905834198,0.8512510925074565,1.6083178520202637,139.0,7.744000244140626,3.9200000762939453,120.55000305175781,240.989911641826,241.989911641867,2,0.6641514301300049,0.10801005662009627,0.409245103597641,0.15068353249336164,3.1094194409255347
60,J012924-733153,J012924-733153_179,22.350415,-73.531439,B,18.4689998626709,0.00013900268822908401,0.21099784313347633,1.3799264430999756,1.4701763786530915,1.5586347579956055,53.0,3.808000183105469,3.2680000305175785,142.8000030517578,179.78932071129887,180.78932071127315,2,0.2738746702671051,0.23944895698410754,1.2950847148895264,0.6282359345160137,3.032484954960196


In [11]:
tau = []
od = []
for row in abs_table:
    if row['min_optical_depth'] < 0:
        min_tau = row['peak_tau'] - row['e_peak_tau']
        if min_tau < 0:
            min_tau = 0
        tau_val = '$>{:.2f}$'.format(min_tau)
    else:
        tau_val = '${:.2f}\pm{:.2f}$'.format(row['peak_tau'], row['e_peak_tau'])
    tau.append(tau_val)
    od_val = '${:.2f}\pm{:.2f}$'.format(row['min_optical_depth'], row['e_min_optical_depth'])
    od.append(od_val)
abs_table.add_column(Column(data=tau, name='tau'))
abs_table.add_column(Column(data=od, name='optical_depth'))

In [12]:
abs_sample = np.isin(abs_table['id'], spec_table[sample]['id'])
abs_sample_table = abs_table[abs_sample]

In [13]:
abs_sample_table.sort(['comp_name', 'abs_name'])

# Select the columns to be output
abs_sample_table.keep_columns(['comp_name', 'abs_name', 'start_vel', 'end_vel', 'length', 'optical_depth', 'tau', 'max_sigma'])
abs_sample_table = abs_sample_table['comp_name', 'abs_name', 'start_vel', 'end_vel', 'length', 'optical_depth', 'tau', 'max_sigma']
abs_sample_table

comp_name,abs_name,start_vel,end_vel,length,optical_depth,tau,max_sigma
,,km / s,km / s,chan,,,
str32,str32,float64,float64,int64,str14,str13,float64
J004956-723554,J004956-723554_-2,-2.3081441316237323,-0.3081441316751298,3,$0.84\pm0.04$,$0.18\pm0.05$,4.056181693914019
J004956-723554,J004956-723554_137,137.6918558647783,150.69185586444416,14,$0.49\pm0.05$,$0.72\pm0.10$,10.074527449013305
J004956-723554,J004956-723554_155,155.69185586431564,160.69185586418718,6,$0.55\pm0.05$,$0.60\pm0.09$,8.70680109815613
J004956-723554,J004956-723554_3,3.6918558682220755,4.6918558681963765,2,$0.82\pm0.04$,$0.19\pm0.05$,4.6257749581907825
J005218-722708,J005218-722708_0,-0.41138642493699445,1.58861357514502,3,$0.87\pm0.02$,$0.14\pm0.03$,5.778137024487307
J005218-722708,J005218-722708_153,153.588613581377,159.58861358162304,7,$0.80\pm0.03$,$0.22\pm0.04$,6.52598093889628
J005218-722708,J005218-722708_161,161.588613581705,162.58861358174602,2,$0.88\pm0.03$,$0.13\pm0.03$,4.036366919708395
J005337-723144,J005337-723144_102,102.60935854517776,107.60935854504928,6,$0.73\pm0.06$,$0.31\pm0.07$,4.690425910935347


In [14]:
# Set up formatting of the output table

# Column titles
names = ['Source', 'Feature', 'Min Velocity', 'Max Velocity', 'Width', 'Peak Optical', r'Peak $\tau$', 'Significance']

# Note the names need to be the ones in the table output, as defined above
formats={'Source': lambda x: x.replace('-', '$-$'), 'Feature': lambda x: x.replace('-', '$-$').replace('_', '\_'), 
         'Min Velocity': '$%.1f$', 'Max Velocity': '$%.1f$', 'Significance' : '%.1f'}

# We have to set the units manually to get the brackets around the units
units = {'Min Velocity':r'(kms$^{-1}$)', 'Max Velocity':r'(kms$^{-1}$)', 'Width':r'(kms$^{-1}$)', 
         'Significance': '(sigma)', 'Peak Optical':'Depth'}

col_align=''.join(['l' if name in ('Source', 'Feature') else 'r'  for name in names])
latextdict = {'units': units, 'tabletype': None, 'header_start': '\\hline \\hline', 
              'header_end': '\\hline', 'data_end': '\\hline'}

In [15]:

ascii.write(abs_sample_table, 'absorption_table.tex', Writer=ascii.Latex, latexdict=latextdict, 
            formats=formats, names=names, col_align=col_align, overwrite=True)


### Detections stats

In [16]:
spec_table

id,comp_name,ra,dec,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,has_mw_abs,has_other_abs,semi_maj_axis,semi_min_axis,pa,nh_gass,noise_flag,beam_power,nh_uncorr,e_nh_uncorr,nh_corr,r_hi,f_int,mean_ts,ew,num_chan,tau
,,deg,deg,,mJy/beam,mJy/beam,,,,,,,km / s,,,,,arcsec,arcsec,deg,1 / cm2,,,1 / cm2,1 / cm2,1 / cm2,,,,km / s,,
int64,str32,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,float64,float64,float64,float64,bool,float32,float64,float64,float64,float64,float64,float64,float32,int64,str13
1,J004725-712727,11.85596,-71.457707,B,15.015000343322754,0.0001577413873746991,0.20037295013116518,1.1125783920288086,1.0923846748301125,1.5317274332046509,43.0,0.41914910078048706,96.17224896304,0.869528591632843,0.3925101382105877,False,False,3.82400016784668,3.2720001220703128,133.6300048828125,6.174081421437779e+20,False,1.0359205,5.365187210776425e+20,1.986559023022372e+19,5.917234854795377e+20,1.1028943860356109,1.1028943860356109,49.397795872342485,7.715428,40,$0.87\pm0.39$
5,J002620-743741,6.586833,-74.628167,C,15.201000213623047,0.00014202001038938762,0.26400873561229615,1.9746829271316528,1.4209718128364035,1.815657138824463,67.0,-0.15902580320835114,132.02632915333402,5.0,3.8316940181297285,False,False,4.056000137329102,3.3919998168945313,135.64999389648438,1.3315663738275294e+21,False,0.88999236,1.662447479255305e+21,4.117855118886791e+19,1.8430605679293883e+21,1.1086428840175986,1.1086428840175986,125.01606621274738,13.0891905,75,$>1.17$
9,J011635-712604,19.148794,-71.434582,C,15.333000183105469,0.0001255630166269839,0.21174872823938362,1.6042497158050537,0.7604905305141251,1.9112515449523926,74.0,0.30700182914733887,185.81443163211898,1.1809015274047852,0.6544683758159434,False,False,3.8040000915527346,3.2080001831054688,142.02000427246094,1.227406986785527e+21,False,1.0161469,9.243698933001397e+20,2.9086163229354488e+19,9.876735081156763e+20,1.0684829907100641,1.0684829907100641,169.7816602456293,6.9796023,85,$1.18\pm0.65$
10,J010425-742733,16.10635,-74.459193,C,15.435999870300293,0.00011435524793341756,0.2508074329566693,1.681606411933899,0.8255535251261913,1.9211487770080566,78.0,0.2395423799753189,75.45870833663803,1.4290249347686768,0.7164151873632499,False,False,3.7599998474121095,3.2759998321533206,139.1699981689453,3.131320668910356e+20,False,0.95265865,--,0.0,--,--,--,0.0,0.0,0,$1.43\pm0.72$
13,J005535-723533,13.899411,-72.592574,C,15.053000450134277,8.522021817043423e-05,0.16227310227980046,1.856298565864563,2.5452221979121297,1.5236057043075562,93.0,-0.33269286155700684,113.61597766529509,5.0,3.6192534660730047,False,True,5.552000045776367,4.431999969482422,140.7100067138672,7.446596934360939e+21,False,0.9928809,7.168075769842141e+21,6.882111346062339e+19,8.849933566255248e+21,1.2346316989964163,1.2346316989964163,479.813134319267,19.196941,94,$>1.38$
14,J012336-703232,20.902153,-70.542329,C,15.822999954223633,0.00015618808101862668,0.24878249600046518,1.6433773040771484,0.9665233841956862,1.8356764316558838,55.0,0.19229918718338013,225.31028406877002,1.64870285987854,0.8308259887378207,False,False,3.9840000152587893,3.2560001373291017,123.41999816894531,4.156697450338526e+20,False,1.0374693,2.8895810711429556e+20,1.4532738940086557e+19,2.88870179026072e+20,0.9996957064499707,0.9996957064499707,-110.38500318620588,-0.62251383,28,$1.65\pm0.83$
15,J003530-723655,8.877739,-72.615448,C,16.052000045776367,0.0001357692526653409,0.26143587233583027,1.6275757551193237,0.8721049797536266,1.8693827390670776,72.0,0.2418069839477539,235.912981704439,1.4196155071258545,0.7330660214610306,False,False,3.808000183105469,3.2680000305175785,134.50999450683594,9.376269000777239e+20,False,0.9938403,8.317796077639564e+20,2.5192293467713835e+19,8.449818303161053e+20,1.0158722604268213,1.0158722604268213,-388.8760488258296,1.6447271,54,$1.42\pm0.73$
17,J010524-731518,16.350785,

In [17]:
rows= []
num_recs = len(spec_table)
num_det = np.sum(spec_table['has_other_abs'])
rate = num_det / num_recs * 100
print ("Found {} detections from {} sources ({:.0f} %)".format(num_det, num_recs, rate))
non_dets = spec_table[~spec_table['has_other_abs']]
max_non_det = np.max(non_dets['flux_peak'])
print ("Non detections - max peak flux {:.2f} lowest noise {:.3f}".format(max_non_det, np.min(non_dets['sd_cont'])))
#print ("{} & {} & {:.0f} \\".format(num_recs, num_det, rate))
rows.append(("All",num_recs, num_det, rate))

Found 65 detections from 229 sources (28 %)
Non detections - max peak flux 423.12 lowest noise 0.009


In [18]:
smc_sample = spec_table[spec_table['nh_uncorr'] >= 2e21]
num_smc_recs = len(smc_sample)
num_smc_det = np.sum(smc_sample['has_other_abs'])
smc_rate = num_smc_det / num_smc_recs * 100
print ("Found {} detections from {} SMC sources ({:.0f} %)".format(num_smc_det, num_smc_recs, smc_rate))

non_dets = smc_sample[~smc_sample['has_other_abs']]
max_non_det = np.max(non_dets['flux_peak'])
print ("Non detections - max peak flux {:.2f} lowest noise {:.3f}".format(max_non_det, np.min(non_dets['sd_cont'])))

#print ("{} & {} & {:.0f} \\".format(num_smc_recs, num_smc_det, smc_rate))
rows.append(("SMC Body",num_smc_recs, num_smc_det, smc_rate))

Found 49 detections from 79 SMC sources (62 %)
Non detections - max peak flux 38.45 lowest noise 0.073


/Users/jamesdempsey/anaconda/envs/python3/lib/python3.6/site-packages/astropy/table/column.py:991: RuntimeWarning: invalid value encountered in greater_equal
  result = getattr(super(), op)(other)


In [19]:
non_smc_sample = spec_table[spec_table['nh_uncorr'] < 2e21]
num_non_smc_recs = len(non_smc_sample)
num_non_smc_det = np.sum(non_smc_sample['has_other_abs'])
non_smc_rate = num_non_smc_det / num_non_smc_recs * 100
print ("Found {} detections from {} non-SMC sources ({:.0f} %)".format(num_non_smc_det, num_non_smc_recs, non_smc_rate))

non_dets = non_smc_sample[~non_smc_sample['has_other_abs']]
max_non_det = np.max(non_dets['flux_peak'])
print ("Non detections - max peak flux {:.2f} lowest noise {:.3f}".format(max_non_det, np.min(non_dets['sd_cont'])))

#print ("{} & {} & {:.0f} \\".format(num_non_smc_recs, num_non_smc_det, non_smc_rate))
rows.append(("Outside SMC",num_non_smc_recs, num_non_smc_det, non_smc_rate))

Found 16 detections from 134 non-SMC sources (12 %)
Non detections - max peak flux 423.12 lowest noise 0.009


/Users/jamesdempsey/anaconda/envs/python3/lib/python3.6/site-packages/astropy/table/column.py:991: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)


In [20]:
for row in rows:
    print ("{} & {} & {} & {:.0f} \\\\".format(row[0], row[1], row[2], row[3]))

All & 229 & 65 & 28 \\
SMC Body & 79 & 49 & 62 \\
Outside SMC & 134 & 16 & 12 \\
